# Example plots

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import subjects
from neuropy import plotting
from neuropy.utils import signal_process
from neuropy.analyses import Pf1D
from neuropy.analyses import Decode1d

sessions = subjects.sd.ratSday3

In [ ]:
for sub,sess in enumerate(sessions):
    maze = sess.paradigm['maze']
    post = sess.paradigm['post']
    neurons = sess.neurons.get_neuron_type(neuron_type='pyr')
    pos = sess.lin_maze
    pf = Pf1D(neurons=neurons,position=pos,speed_thresh=3,smooth=2,grid_bin=3)
    pf_neurons =neurons.get_by_id(pf.ratemap.neuron_ids) 
    rpls = sess.pbe.time_slice(maze[0],maze[1])
    decode = Decode1d(neurons=pf_neurons,ratemap = pf.ratemap,epochs=rpls,bin_size=0.010)

In [ ]:
decode.calculate_shuffle_score(method='neuron_id',n_iter=100)


In [44]:
from hfuncs import plot_in_bokeh
import bokeh.plotting as bplot

bplot.output_file(subjects.figpath_sd/'test_decoding.html')
p = plot_in_bokeh(x=rpls.starts,y=decode.score,img_arr=decode.posterior,color_by=decode.score,palette='Blues')
p.line(pos.time,pos.x/250 + 0.2,line_width=2,color='black')
bplot.show(p)

Opening in existing browser session.


In [ ]:
from matplotlib import cm


In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm

dirpath = subjects.figpath_sd/'decoded'
# for i,evt in enumerate(decode.posterior):
evt = decode.posterior[200]
cmap =cm.get_cmap('hot') 
im_cmap =cmap(evt)
im_arr = Image.fromarray(evt)
# k = plt.pcolormesh(img_as_ubyte(im_arr),cmap='hot')
plt.imshow(evt,aspect='auto')
    # plt.savefig(f'test{i}.png')


In [ ]:
from bokeh.plotting import ColumnDataSource, figure, output_notebook, show,output_file
import numpy as np
from PIL import Image
from io import BytesIO
from matplotlib import cm
# output_notebook()
output_file(subjects.figpath_sd/'test_decoded.html')
from base64 import b64decode,b64encode,b32encode
import matplotlib.pyplot as plt
from skimage import img_as_ubyte
import matplotlib as mpl
from bokeh.models import ColorBar,LinearColorMapper

score = decode.score
def to_png(arr):
    out = BytesIO()
    # print(arr.shape)
    im = Image.fromarray(arr)
    im.save(out, format='png')
    return out.getvalue()

def b64_image_files(images, colormap='hot'):
    cmap = cm.get_cmap(colormap)
    urls = []
    for im in images:
        # im_ = cmap(((np.flipud(im))/np.max(im)))
        im_ = (im-np.min(im))/np.max(im)
        # im_ = np.where(im_>0.2,im_,0)
        im_ = cmap(im_)
        png = to_png(img_as_ubyte(im_))
        url = 'data:image/png;base64,' + b64encode(png).decode('utf-8')
        urls.append(url)
    return urls


# im = Image.fromarray(score)
# im.save()

out = BytesIO()
a = np.arange(25).reshape(5,5)
a = a/25
# encoded = b64decode(a)
# mime = "image/jpeg"

filenames = b64_image_files(decode.posterior)
# uri = "data:%s;base64,%s" % (mime, encoded)
# im = plt.imshow(a)

# im = Image.fromarray(a)
# im.save(out,format='png')

# im = Image.fromarray(np.uint8(cm.gist_earth(a)*255))
# img = out.getvalue()

cmap = cm.get_cmap('jet_r')
# pval = decode.p_value
# pval = np.where(pval>0.9,pval,0)
col =cmap(decode.score) 
# col = [
#     "#%02x%02x%02x" % (int(r), int(g), int(b)) for r, g, b, _ in 255*mpl.cm.viridis(mpl.colors.Normalize()(score))
# ]
# col = [_ for _ in col]
source = ColumnDataSource(data=dict(
    x=rpls.starts,
    y=score,
    desc=decode.score,
    imgs=filenames,
    colors = col,
 
))

TOOLTIPS = """
    <div>
        <div>
            <img
                src="@imgs" height="100" alt="@imgs" width="100"
                style="float: left; margin: 0px 15px 15px 0px;image-rendering: pixelated"
                border="2"
            ></img>
        </div>
        <div>
            <span style="font-size: 17px; font-weight: bold;">p_val @desc</span>
            <span style="font-size: 17px; font-weight: bold;">, score $y</span>
        </div>
        <div>
            <span style="font-size: 15px; color: #696;">time $x</span>
        </div>
    </div>
"""

p = figure(width=1800, height=800, tooltips=TOOLTIPS,x_axis_label='Time',y_axis_label='Replay score',
           title="Replay score over time, Hover to see posterior")
p.circle('x', 'y', size=12, source=source,color='colors')
# p.scatter(rpls.starts,score,fill_color = col)
# p.line(pos.time,pos.x/250 + 0.2,line_width=2,color='black')
col1 = LinearColorMapper(palette = "Cividis256",
                          low = 0, high = 1)
cb = ColorBar(color_mapper = col1, location = (1850,6))
p.add_layout(cb)
show(p)

In [ ]:
%matplotlib widget
from scipy.ndimage import gaussian_filter1d
# plt.pcolormesh(decode.posterior[2],cmap='hot')
# plt.plot(rpls.starts,score,'.')
plt.plot(pos.speed)
# plt.plot(gaussian_filter1d(pos.speed,sigma=20))

In [ ]:
%matplotlib widget
from neuropy.plotting import plot_position
from scipy.ndimage import gaussian_filter1d

smooth_pos = gaussian_filter1d(pos.traces[0],sigma=10)
plt.plot(smooth_pos)
plt.plot(pos.traces[0],'gray')

In [ ]:
%matplotlib widget

plt.plot(np.abs(np.diff(smooth_pos)))

In [ ]:
max(pos.x)

In [ ]:
%matplotlib widget
from neuropy.plotting import plot_ratemap

plot_ratemap(pf.ratemap,normalize_tuning_curve=True)

In [ ]:
%matplotlib inline

plt.plot(pf.ratemap.tuning_curves[20])